In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
nu_fe_sa = pd.read_csv("../output/number_feature_sample.tsv", "\t", index_col=0)

bi_co_sa = pd.read_csv("../output/01_comparison_sample.tsv", "\t", index_col=0)

In [ ]:
def get_log_ratio(bi_, lo_):

    return lo_[bi_ == 1].mean() - lo_[bi_ == 0].mean()


get_log_ratio(np.array([0, 0, 1, 1]), np.array([10, 20, 30, 40]))

In [ ]:
ke = {
    "tyta": "binary",
    "st": SETTING["st"],
}

for co, ta in bi_co_sa.iterrows():

    ta.dropna(inplace=True)

    simi = np.unique(ta, return_counts=True)[1].min()

    for fu, sire in [
        [get_log_ratio, 1],
        [kraft.information.get_ic, 2],
    ]:

        if simi < sire:

            continue

        funa = fu.__name__

        pa = "../output/compare_feature/{}/{}/".format(kraft.path.clean(co), funa)

        kraft.path.make(pa)

        st = kraft.function_heat_map.make(
            ta,
            nu_fe_sa,
            fu,
            n_jo=SETTING["n_jo"],
            n_sa=SETTING["n_sa"],
            n_sh=SETTING["n_sh"],
            n_pl=SETTING["n_ex"],
            title="All ({})".format(funa),
            pa=pa,
            **ke,
        )

        if 0 < len(SETTING["fe_"]):

            kraft.function_heat_map.make(
                ta,
                nu_fe_sa.reindex(SETTING["fe_"]),
                st,
                n_pl=None,
                title="Peek ({})".format(funa),
                **ke,
            )

            plot_peek(st["Score"], SETTING["fe_"], pa)